
# AI Agentic Design Patterns with AutoGen

This notebook assists to build and customize multi-agent systems, enabling agents to take on different roles and collaborate to accomplish complex tasks using AutoGen, a framework that enables the development of LLM applications using multi-agents.

## 📋 Table of Contents

1. **Introduction to Agentic Architectures**
   - Overview of Agentic Design Patterns: Understanding the foundational concepts of agentic architectures and their significance in AI.

2. **Building Multi-Agent Systems**
   - Creating Conversable Agents: Learning to construct multi-agent conversations using “ConversableAgent,” a built-in agent class of AutoGen.

3. **Multi-Agent Collaboration Design Patterns**
   - Collaborative Design Patterns: Implementing patterns for multi-agent collaboration to handle complex tasks.

4. **Agent Reflection Framework**
   - Developing High-Quality Content: Utilizing the agent reflection framework to create and review content.

5. **Tool Use Design Pattern**
   - Integrating Tools with Agents: Implementing design patterns that allow agents to call tools and perform specific tasks.

7. **Tracing Options**
   - Understanding the methods available for tracing agent activities and decisions to improve transparency and debuggability.

6. **Multi-Agent Architecture for Document Classification**
   - **Data Aggregation from Multiple Sources**: Designing a system where agents are responsible for gathering data from various sources. Each agent specializes in retrieving information from a specific source, ensuring comprehensive data collection.
   - **Collaborative Information Analysis**: Implementing a framework where agents share and analyze the collected data. This step involves preprocessing, normalization, and synthesis of information to prepare for classification.
   - **Decision-Making with SME (Subject Matter Expert) Agents**: Integrating SME agents that utilize the aggregated information to classify documents. These agents apply domain-specific knowledge and criteria to make informed decisions.
   - **Input and Output Management**: Establishing protocols for efficient data input from multiple sources and outputting the classification results. This includes defining data formats, communication standards, and result dissemination methods.

For more information on AutoGen, including tutorials and documentation, visit the following resources:

- [AutoGen GitHub Repository](https://github.com/microsoft/autogen) - Find the source code, examples, and more.
- [AutoGen Documentation](https://microsoft.github.io/autogen/docs/tutorial/introduction) - Explore detailed tutorials, API references, and an introduction to AutoGen.

In [27]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Set the service endpoint and API key from the environment
# Create an SDK client
endpoint = os.environ["AZURE_AI_SEARCH_SERVICE_ENDPOINT"]

admin_index_client = SearchIndexClient(
    endpoint=endpoint,
    index_name=os.environ["SEARCH_INDEX_NAME"],
    credential=AzureKeyCredential(os.environ["AZURE_SEARCH_ADMIN_KEY"]),
)

In [1]:
from src.tools.pubmed import PubMedScraper

pb = PubMedScraper(email="pablosalvadorlopez11@gmail.com")
df = pb.search_by_query(query="covid-19 in spain", max_results=10)

In [5]:
df['full_content'][0] == df['full_content'][1]

False

In [6]:
len(df['full_content'][1])

6075

In [7]:
len(df['full_content'][0])

68161

In [55]:
import requests
import xml.etree.ElementTree as ET
import json
import pandas as pd
from typing import List, Dict, Any, Optional
from bs4 import BeautifulSoup
import fitz  # PyMuPDF
from Bio import Entrez
import time

Entrez.email = "pablosalvadorlopez11@gmail.com"  # Replace with your email

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

def fetch_pubmed_ids(query: str, max_results: int = 10, field: str = None, sort: str = "relevance", 
                     datetype: str = None, mindate: str = None, maxdate: str = None, 
                     retstart: int = 0) -> List[str]:
    """
    Fetch PubMed IDs based on a search query.

    Parameters:
    query (str): The search query for PubMed.
    max_results (int): The maximum number of results to return.
    field (str): The field to search (e.g., "title", "author", "journal").
    sort (str): The sort order of the results (e.g., "relevance", "pub date").
    datetype (str): The date type to filter results (e.g., "pdat" for publication date).
    mindate (str): The minimum date for filtering results.
    maxdate (str): The maximum date for filtering results.
    retstart (int): The starting index of the results for pagination.

    Returns:
    List[str]: A list of PubMed IDs.
    """
    base_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi"
    params = {
        "db": "pubmed",
        "term": query,
        "retmax": max_results,
        "sort": sort,
        "retmode": "json",
        "retstart": retstart
    }
    
    if field:
        params["field"] = field
    if datetype:
        params["datetype"] = datetype
    if mindate:
        params["mindate"] = mindate
    if maxdate:
        params["maxdate"] = maxdate

    response = requests.get(base_url, params=params, headers=HEADERS)
    response.raise_for_status()  # Ensure we handle HTTP errors
    data = response.json()
    return data.get('esearchresult', {}).get('idlist', [])

def fetch_pubmed_articles(ids: List[str]) -> str:
    """
    Fetch PubMed article details using IDs.

    Parameters:
    ids (List[str]): List of PubMed IDs.

    Returns:
    str: XML data containing the article details.
    """
    handle = Entrez.efetch("pubmed", id=",".join(ids), retmode="xml")
    return handle.read()

def parse_article_details(xml_data: str) -> List[Dict[str, Any]]:
    """
    Parse PubMed article details from XML data, enhanced to include PMID, Year, Volume, Issue, and Citation.
    """
    root = ET.fromstring(xml_data)
    articles = []
    for article in root.findall(".//PubmedArticle"):
        pmid = article.findtext(".//PMID", default="No PMID available")
        year = article.findtext(".//PubDate/Year", default="No Year available")
        volume = article.findtext(".//JournalIssue/Volume", default="No Volume available")
        issue = article.findtext(".//JournalIssue/Issue", default="No Issue available")
        citation = f"{year};{volume}({issue})"
        
        details = {
            'pmid': pmid,
            'title': article.findtext(".//ArticleTitle", default="No title available"),
            'abstract': article.findtext(".//AbstractText", default="No abstract available"),
            'authors': [
                f"{author.findtext('ForeName', '')} {author.findtext('LastName', '')}".strip() 
                for author in article.findall(".//Author")
            ],
            'year': year,
            'volume': volume,
            'issue': issue,
            'journal': article.findtext(".//Journal/Title", default="No journal available"),
            'citation': citation,
            'link': article.findtext(".//ArticleId[@IdType='doi']", default="No link available")
        }
        if details['link'] and details['link'] != "No link available":
            details['link'] = f"https://doi.org/{details['link']}"
        
        pmc_id = None
        for other_id in article.findall(".//ArticleIdList/ArticleId"):
            if other_id.attrib.get('IdType') == 'pmc':
                pmc_id = other_id.text
                break
        if pmc_id:
            details['pdf_link'] = f"https://www.ncbi.nlm.nih.gov/pmc/articles/{pmc_id}/pdf/"
        else:
            details['pdf_link'] = "No PDF link available"
        
        articles.append(details)
    return articles

def articles_to_json(articles: List[Dict[str, Any]]) -> str:
    """
    Convert articles metadata to JSON format.

    Parameters:
    articles (List[Dict[str, Any]]): A list of dictionaries containing article metadata.

    Returns:
    str: JSON formatted string.
    """
    return json.dumps(articles, indent=4)

def fetch_pdf_content(url: str) -> Optional[str]:
    """
    Fetch the full content of a PDF article from a given URL.

    Parameters:
    url (str): URL of the PDF article.

    Returns:
    Optional[str]: The text content of the PDF or an error message if fetching fails.
    """
    retries = 3
    for _ in range(retries):
        try:
            response = requests.get(url, headers=HEADERS)
            response.raise_for_status()  # Ensure we handle HTTP errors
            with open('temp_article.pdf', 'wb') as f:
                f.write(response.content)
            
            # Extract text from the PDF
            pdf_document = fitz.open('temp_article.pdf')
            text = ""
            for page in pdf_document:
                text += page.get_text()
            return text
        except requests.HTTPError as e:
            if e.response.status_code == 403:
                time.sleep(2)
                continue  # Retry on 403 error
            return f"Error fetching PDF content: {e}"
    return "Failed to fetch PDF content after retries."

def add_article_content(articles: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    """
    Add full content to each article's metadata.

    Parameters:
    articles (List[Dict[str, Any]]): A list of dictionaries containing article metadata.

    Returns:
    List[Dict[str, Any]]: A list of dictionaries with added full content.
    """
    for article in articles:
        if 'pdf_link' in article and article['pdf_link'] != "No PDF link available":
            article['full_content'] = fetch_pdf_content(article['pdf_link'])
        else:
            article['full_content'] = "No full text link available"
    return articles

def json_to_csv(json_data: str, csv_file_path: str = None) -> pd.DataFrame:
    """
    Convert JSON data containing PubMed articles to a CSV file with organized columns,
    or return a DataFrame if no CSV file path is provided.

    Parameters:
    json_data (str): JSON formatted string containing article metadata.
    csv_file_path (str, optional): Path where the CSV file will be saved. If not provided, the function returns a DataFrame.

    Returns:
    pd.DataFrame: DataFrame containing the article metadata if no csv_file_path is provided.
    """
    articles = json.loads(json_data)
    columns = ['pmid', 'title', 'abstract', 'authors', 'year', 'volume', 'issue', 'journal', 'citation', 'link', 'pdf_link', 'full_content']
    df = pd.DataFrame(articles, columns=columns)
    if csv_file_path:
        df.to_csv(csv_file_path, index=False)
    return df

# Example usage

article_ids = fetch_pubmed_ids("Alzheimer's treatment in norway", max_results=5)
xml_data = fetch_pubmed_articles(article_ids)
article_details = parse_article_details(xml_data)
article_details_with_content = add_article_content(article_details)
json_output = articles_to_json(article_details_with_content)
df_result = json_to_csv(json_output)
df_result


,pmid,title,abstract,authors,year,volume,issue,journal,citation,link,pdf_link,full_content
0,30746447,,,"[Stephen S Dominy, Casey Lynch, Florian Ermini...",2019,5,1,Science advances,2019;5(1),https://doi.org/10.1126/sciadv.aau3333,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6...,"Dominy et al., Sci. Adv. 2019; 5 : eaau3333 ..."
1,35277027,The Potential Role of Gut Microbiota in Alzhei...,Gut microbiota is emerging as a key regulator ...,"[Angelica Varesi, Elisa Pierella, Marcello Rom...",2022,14,3,Nutrients,2022;14(3),https://doi.org/10.3390/nu14030668,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8...,"\n\nCitation: Varesi, A.; Pie..."
2,34497121,NAD,Alzheimer's disease (AD) is a progressive and ...,"[Yujun Hou, Yong Wei, Sofie Lautrup, Beimeng Y...",2021,118,37,Proceedings of the National Academy of Science...,2021;118(37),https://doi.org/10.1073/pnas.2011226118,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8...,NAD+ supplementation reduces neuroinflammation...
3,36087307,Head-to-head comparison of 10 plasma phospho-t...,Plasma phospho-tau (p-tau) species have emerge...,"[Shorena Janelidze, Divya Bali, Nicholas J Ash...",2023,146,4,Brain : a journal of neurology,2023;146(4),https://doi.org/10.1093/brain/awac333,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,https://doi.org/10.1093/brain/awac333\nBRAIN 2...
4,35851957,The promise of music therapy for Alzheimer's d...,Alzheimer's disease (AD) is a progressive neur...,"[Anna Maria Matziorinis, Stefan Koelsch]",2022,1516,1,Annals of the New York Academy of Sciences,2022;1516(1),https://doi.org/10.1111/nyas.14864,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC9...,DOI: 10.1111/nyas.14864\nC O N C I S E R E V I...


In [ ]:
import pandas as pd
from metapub import PubMedFetcher

keyword = "Alzheimer's treatment"
num_of_articles = 100

fetch = PubMedFetcher()
pmids = fetch.pmids_for_query(keyword, retmax=num_of_articles)
articles = {}

# Retrieve information for each article
for pmid in pmids:
    articles[pmid] = fetch.article_by_pmid(pmid)

# Extract relevant information and create DataFrames
titles = {}
for pmid in pmids:
    titles[pmid] = fetch.article_by_pmid(pmid).title
Title = pd.DataFrame(list(titles.items()), columns=['pmid', 'Title'])

# Similar steps for abstracts, authors, years, volumes, issues, journals, citations, and links

# Merge all DataFrames into a single one
data_frames = [Title, Abstract, Author, Year, Volume, Issue, Journal, Citation, Link]
from functools import reduce
df_merged = reduce(lambda  left, right: pd.merge(left, right, on=['pmid'], how='outer'), data_frames)

In [35]:
import pandas as pd
from metapub import PubMedFetcher

keyword = "Alzheimer's treatment in norway"
num_of_articles = 5

fetch = PubMedFetcher()
pmids = fetch.pmids_for_query(keyword, retmax=num_of_articles)

# Initialize a dictionary to hold all article information
articles_info = {'pmid': [], 'Title': [], 'Abstract': [], 'Author': [], 'Year': [], 'Volume': [], 'Issue': [], 'Journal': [], 'Citation': [], 'Link': []}

# Retrieve information for each article and populate the dictionary
for pmid in pmids:
    article = fetch.article_by_pmid(pmid)
    articles_info['pmid'].append(pmid)
    articles_info['Title'].append(article.title)
    articles_info['Abstract'].append(article.abstract)
    articles_info['Author'].append(article.authors)
    articles_info['Year'].append(article.year)
    articles_info['Volume'].append(article.volume)
    articles_info['Issue'].append(article.issue)
    articles_info['Journal'].append(article.journal)
    articles_info['Citation'].append(article.citation)
    articles_info['Link'].append(f"https://doi.org/{article.doi}" if article.doi else None)

# Convert the dictionary to a DataFrame
df_merged = pd.DataFrame(articles_info)

In [36]:
article = fetch.article_by_pmid("38876257")

In [47]:
!pip install biopython

   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.7 MB 660.6 kB/s eta 0:00:05
   ---- ----------------------------------- 0.3/2.7 MB 3.5 MB/s eta 0:00:01
   ----------------------------- ---------- 2.0/2.7 MB 16.0 MB/s eta 0:00:01
   ---------------------------------------- 2.7/2.7 MB 17.4 MB/s eta 0:00:00


In [48]:
from Bio import Entrez

In [49]:
def download_article(pubmed_id):
    Entrez.email = "your_email@example.com"  # Always tell NCBI who you are
    handle = Entrez.efetch(db="pubmed", id=pubmed_id, rettype="full", retmode="xml")
    article_data = handle.read()
    handle.close()
    
    return article_data

pubmed_id = "38876257"
article_data = download_article(pubmed_id)

# article_data now contains the XML data as a string
# You can use article_data as needed, it's stored in memory

In [50]:
article_data

b'<?xml version="1.0" ?>\n<!DOCTYPE PubmedArticleSet PUBLIC "-//NLM//DTD PubMedArticle, 1st January 2024//EN" "https://dtd.nlm.nih.gov/ncbi/pubmed/out/pubmed_240101.dtd">\n<PubmedArticleSet>\n<PubmedArticle><MedlineCitation Status="Publisher" Owner="NLM" IndexingMethod="Automated"><PMID Version="1">38876257</PMID><DateRevised><Year>2024</Year><Month>06</Month><Day>14</Day></DateRevised><Article PubModel="Print-Electronic"><Journal><ISSN IssnType="Electronic">1872-9649</ISSN><JournalIssue CitedMedium="Internet"><PubDate><Year>2024</Year><Month>Jun</Month><Day>12</Day></PubDate></JournalIssue><Title>Ageing research reviews</Title><ISOAbbreviation>Ageing Res Rev</ISOAbbreviation></Journal><ArticleTitle>Identification and potential clinical applications of novel autophagy/mitophagy proteins in the biofluids of Alzheimer\'s disease patients.</ArticleTitle><Pagination><StartPage>102378</StartPage><MedlinePgn>102378</MedlinePgn></Pagination><ELocationID EIdType="doi" ValidYN="Y">10.1016/j.arr

In [46]:
article.to_dict()

{'pubmed_type': 'article',
 '_root': 'MedlineCitation',
 'xml': b'<?xml version="1.0" ?>\n<!DOCTYPE PubmedArticleSet PUBLIC "-//NLM//DTD PubMedArticle, 1st January 2024//EN" "https://dtd.nlm.nih.gov/ncbi/pubmed/out/pubmed_240101.dtd">\n<PubmedArticleSet>\n<PubmedArticle><MedlineCitation Status="Publisher" Owner="NLM" IndexingMethod="Automated"><PMID Version="1">38876257</PMID><DateRevised><Year>2024</Year><Month>06</Month><Day>14</Day></DateRevised><Article PubModel="Print-Electronic"><Journal><ISSN IssnType="Electronic">1872-9649</ISSN><JournalIssue CitedMedium="Internet"><PubDate><Year>2024</Year><Month>Jun</Month><Day>12</Day></PubDate></JournalIssue><Title>Ageing research reviews</Title><ISOAbbreviation>Ageing Res Rev</ISOAbbreviation></Journal><ArticleTitle>Identification and potential clinical applications of novel autophagy/mitophagy proteins in the biofluids of Alzheimer\'s disease patients.</ArticleTitle><Pagination><StartPage>102378</StartPage><MedlinePgn>102378</MedlinePgn><

In [34]:
df_merged

,pmid,Title,Abstract,Author,Year,Volume,Issue,Journal,Citation,Link
0,38876257,Identification and potential clinical applicat...,None,"[Zhang J, Wang HL, Veverová K, Vyhnálek M, Fan...",2024,None,None,Ageing Res Rev,"Zhang J, et al. Identification and potential c...",https://doi.org/10.1016/j.arr.2024.102378
1,38849413,Genome-wide determinants of mortality and moto...,There are 90 independent genome-wide significa...,"[Tan MMX, Lawton MA, Pollard MI, Brown E, Real...",2024,10,1,NPJ Parkinsons Dis,"Tan MMX, et al. Genome-wide determinants of mo...",https://doi.org/10.1038/s41531-024-00729-8
2,38848061,Biomarkers of Neurobiologic Recovery in Adults...,"IMPORTANCE: Sport-related concussion (SRC), a ...","[O'Brien WT, Spitz G, Xie B, Major BP, Mutimer...",2024,7,6,JAMA Netw Open,"O'Brien WT, et al. Biomarkers of Neurobiologic...",https://doi.org/10.1001/jamanetworkopen.2024.1...
3,38838385,Integrating AI in fighting advancing Alzheimer...,The application of artificial intelligence (AI...,"[Angelucci F, Ai AR, Piendel L, Cerman J, Hort J]",2024,87,None,Curr Opin Struct Biol,"Angelucci F, et al. Integrating AI in fighting...",https://doi.org/10.1016/j.sbi.2024.102857
4,38830549,"Donanemab, another anti-Alzheimer's drug with ...","Based on ""reducing amyloid plaques in the brai...","[Høilund-Carlsen PF, Alavi A, Barrio JR, Caste...",2024,None,None,Ageing Res Rev,"Høilund-Carlsen PF, et al. Donanemab, another ...",https://doi.org/10.1016/j.arr.2024.102348


In [2]:
import requests

# Purpose: Retrieve information about clinical studies conducted around the world.

def fetch_clinical_trials(query, max_results=10):
    base_url = "https://clinicaltrials.gov/api/query/study_fields"
    params = {
        "expr": query,
        "fields": "NCTId,BriefTitle,Condition",
        "max_rnk": max_results,
        "fmt": "json"
    }
    response = requests.get(base_url, params=params)
    data = response.json()
    return data['StudyFieldsResponse']['StudyFields']

# Example usage
trials = fetch_clinical_trials("diabetes")
print(trials)


[{'Rank': 1, 'NCTId': ['NCT06419907'], 'BriefTitle': ['Differing Completion Rates of DIABETES Education on Patient Reported Outcomes'], 'Condition': ['Diabetes Mellitus, Type 2', 'Patient Education as Topic']}, {'Rank': 2, 'NCTId': ['NCT04016584'], 'BriefTitle': ['Diabetes Pueblo Program - Application and Acceptability of Culturally Appropriate Latino Education for Insulin Therapy'], 'Condition': ['Type2 Diabetes']}, {'Rank': 3, 'NCTId': ['NCT04216875'], 'BriefTitle': ['Best Practice Study of Diabetes Type 2 Management in Primary Care in Switzerland'], 'Condition': ['Diabetes Mellitus, Type 2', 'Primary Care']}, {'Rank': 4, 'NCTId': ['NCT02076568'], 'BriefTitle': ['Diabetes and Partnership: Evaluation of a Diabetes Education Module'], 'Condition': ['Diabetes Mellitus']}, {'Rank': 5, 'NCTId': ['NCT02076542'], 'BriefTitle': ['Diabetes and Sports: Evaluation of a Diabetes Education Module'], 'Condition': ['Diabetes Mellitus']}, {'Rank': 6, 'NCTId': ['NCT02077686'], 'BriefTitle': ['Diabete

In [5]:
import requests

# Purpose: Access public FDA data, including drug, device, and food recalls, adverse events, and more.

def fetch_openfda_data(endpoint, query, max_results=10):
    base_url = f"https://api.fda.gov/{endpoint}.json"
    params = {
        "search": query,
        "limit": max_results
    }
    response = requests.get(base_url, params=params)
    data = response.json()
    return data['results']

# Example usage
drugs = fetch_openfda_data("drug/event", "aspirin")
print(drugs)


[{'safetyreportversion': '2', 'safetyreportid': '10003304', 'primarysourcecountry': 'US', 'occurcountry': 'US', 'transmissiondateformat': '102', 'transmissiondate': '20141212', 'reporttype': '1', 'serious': '2', 'receivedateformat': '102', 'receivedate': '20140312', 'receiptdateformat': '102', 'receiptdate': '20140424', 'fulfillexpeditecriteria': '2', 'companynumb': 'US-PFIZER INC-2014063856', 'duplicate': '1', 'reportduplicate': {'duplicatesource': 'PFIZER', 'duplicatenumb': 'US-PFIZER INC-2014063856'}, 'primarysource': {'reportercountry': 'US', 'qualification': '1'}, 'sender': {'sendertype': '2', 'senderorganization': 'FDA-Public Use'}, 'receiver': {'receivertype': '6', 'receiverorganization': 'FDA'}, 'patient': {'patientsex': '2', 'reaction': [{'reactionmeddraversionpt': '17.0', 'reactionmeddrapt': 'Drug hypersensitivity', 'reactionoutcome': '6'}], 'drug': [{'drugcharacterization': '1', 'medicinalproduct': 'DOXYCYCLINE HYCLATE', 'drugauthorizationnumb': '050007', 'drugdosagetext': '

In [4]:
# Purpose: Access consumer health information including health topics, drugs, supplements, and more.

import requests

def fetch_medlineplus_data(query):
    base_url = "https://wsearch.nlm.nih.gov/ws/query"
    params = {
        "db": "healthTopics",
        "term": query,
        "retmax": 1,
        "retmode": "json"
    }
    response = requests.get(base_url, params=params)
    data = response.json()
    return data

# Example usage
info = fetch_medlineplus_data("heart disease")
print(info)


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
Purpose: Obtain detailed drug and pharmaceutical information.

import requests

def fetch_drugbank_data(api_key, drug_name):
    base_url = "https://api.drugbank.com/v1/us/drugs"
    headers = {
        "Authorization": f"Bearer {api_key}"
    }
    params = {
        "query": drug_name,
        "limit": 1
    }
    response = requests.get(base_url, headers=headers, params=params)
    data = response.json()
    return data

# Example usage
api_key = "YOUR_DRUGBANK_API_KEY"
drug_info = fetch_drugbank_data(api_key, "aspirin")
print(drug_info)


In [ ]:
import requests

#Purpose: Access biomedical ontologies and related data.

def fetch_bioportal_data(api_key, ontology, term):
    base_url = f"http://data.bioontology.org/search"
    params = {
        "q": term,
        "ontologies": ontology,
        "apikey": api_key
    }
    response = requests.get(base_url, params=params)
    data = response.json()
    return data['collection']

# Example usage
api_key = "YOUR_BIOPORTAL_API_KEY"
bio_data = fetch_bioportal_data(api_key, "SNOMEDCT", "heart disease")
print(bio_data)
